# Cours : Algorithme des $k$-moyennes

Nous allons classifier un ensemble de points issus de trois distributions gaussiennes (de paramètres différents), avec la méthode des $k$-moyennes:

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import random as rd

def points(n):
    tX0 = np.random.multivariate_normal([4, 3], 5*np.eye(2), n) # points de classe 0
    tX1 = np.random.multivariate_normal([-2, -1], 5*np.eye(2), n) # points de classe 1
    tX2 = np.random.multivariate_normal([6, -1], 5*np.eye(2), n) # points de classe 2
    X = np.vstack((tX0, tX1, tX2))
    y = np.hstack((np.zeros(n), np.ones(n)))
    return X, y

X, y = points(100)
plt.scatter(X[:,0], X[:,1]);

k=3

X=X.tolist()
print(X[:2])

[[2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955]]


## Algorithme des $k$ plus proches voisins

In [40]:
def d(x, y):
    s = 0
    for i in range(len(x)):
        s += (x[i] - y[i])**2
    return s**.5

d([2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955])

4.1541833862356015

In [54]:
def centre(classe):
    centre = [0]*len(X[0])
    print(classe)
    for i in range(len(classe)):
        for j in range(len(centre)):
            centre[j] += classe[i][j]
    if len(classe) != 0:
        for j in range(len(centre)):
            centre[j] /= len(classe)
    return centre

centre([[2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955]])

[[2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955]]


[4.377041172780329, -0.7728138779680398]

In [56]:
def calculer_centres(classes):
    centres = []
    for c in classes:
        centres.append(centre(c))
    return centres

centres = calculer_centres([X[:2],X[2:4]])
print(centres)

[[2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955]]
[[7.901517712729385, 1.2978896246566045], [4.713393106263758, 2.4423908888578643]]
[[4.377041172780329, -0.7728138779680398], [6.307455409496571, 1.8701402567572343]]


In [57]:
def plus_proche(x,centres):
    L = [d(x,centres[i]) for i in range(len(centres))]
    min_i=0
    for i in range(len(L)):
        if L[min_i] > L[i]:
            min_i = i
    return(min_i)

plus_proche(X[0],centres)

0

In [59]:
def calculer_classes(X,centres):
    classes = [[] for _ in range(len(centres))]
    for x in X:
        classes[plus_proche(x,centres)].append(x)
    return classes

calculer_classes(X[:10],centres)

[[[2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955], [7.269372737674545, -1.6066146696514014], [3.614438806678783, 1.037048902907393]], [[7.901517712729385, 1.2978896246566045], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [6.535185543874066, 0.5405683584274343], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122]]]

In [48]:
def kmeans(X, centres):
    classes = calculer_classes(X, centres)
    nouveaux_centres = calculer_centres(classes)
    for i in range(len(centres)):
        if not np.array_equal(nouveaux_centres[i], centres[i]):
            return kmeans(X, nouveaux_centres)
    return classes

In [49]:
def heuristique_centres(X,k):
    centres = [X[rd.randint(0,len(X)-1)]]
    for k in range(k-1):        
        max_x=X[0]
        maxi=0
        for x in X:
            dist=0
            for c in centres:
                dist+= d(x,c)
            if dist>maxi:
                maxi=dist
                max_x=x
        centres.append(max_x)
    return centres

heuristique_centres(X,k)

[[7.14249159603677, 2.8251561577834376], [-7.604571680952613, -5.7274142449835805], [6.322882426456804, 7.890235156042933]]

In [36]:
def centres_aleatoires(k):
    np.random.seed(1)
    return np.random.rand(k, len(X[0])).tolist()

In [78]:
res = kmeans(X,centres_aleatoires(3))

[[2.8189720581437347, 0.6007692231345159], [7.901517712729385, 1.2978896246566045], [4.713393106263758, 2.4423908888578643], [7.269372737674545, -1.6066146696514014], [3.2790532147103795, 2.141228355906609], [6.535185543874066, 0.5405683584274343], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.

[[-3.257353167876004, 3.3712402634441343], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-4.765688228146709, 0.9584353795643539], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-3.9259619328279634, 0.5087008366409018], [-2.77141485327454, -1.5161733575595002], [-8.245527927262675, 3.3324461356100743], [-1.7002591202426096, 1.6878763845626845], [-1.363283867676861, -0.4131049500634125], [-0.3032589623094266, 0.9787162630322284], [-5.221896737466192, 1.7554016318552463], [-2.5683634624819724, 2.13014621327879], [-3.7484076748889317, -1.9782998263714955], [-1.7866230182815033, 1.060424991394516], [-1.8641584325458807, -0.5279106960697562], [-1.9630432360019405, -0.6037962126847347], [-4.496502854830695

[[2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955], [7.901517712729385, 1.2978896246566045], [4.713393106263758, 2.4423908888578643], [7.269372737674545, -1.6066146696514014], [3.2790532147103795, 2.141228355906609], [6.535185543874066, 0.5405683584274343], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.

[[-0.1545737498647375, 5.764146803432626], [-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-4.765688228146709, 0.9584353795643539], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-0.5219137024228613, 3.0073875817036413], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-3.9259619328279634, 0.5087008366409018], [-2.77141485327454, -1.5161733575595002], [-8.245527927262675, 3.3324461356100743], [-1.7002591202426096, 1.6878763845626845], [-1.363283867676861, -0.4131049500634125], [-0.13063661783433544, 2.451055885452456], [-0.3032589623094266, 0.9787162630322284], [-3.961661111705337, -2.9404312203496223], [-5.221896737466192, 1.7554016318552463], [-2.5683634624819724,

[[2.8189720581437347, 0.6007692231345159], [5.935110287416922, -2.1463969790705955], [7.901517712729385, 1.2978896246566045], [4.713393106263758, 2.4423908888578643], [7.269372737674545, -1.6066146696514014], [3.2790532147103795, 2.141228355906609], [6.535185543874066, 0.5405683584274343], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.

...

[[-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [0.49183608904818454, -2.3115073976360088], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [1.1473551215894577, -0.7113200917642102], [1.4855951525293154, -0.7553681338709693], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-0.5219137024228613, 3.0073875817036413], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [-2.77141485327454, -1.5161733575595002], [-8.245527927262

[[2.8189720581437347, 0.6007692231345159], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.2188459252399664], [3.5328982940662907, 4.311729332661249], [5.8760239454182335, 5.08200754778629], [4.638592672780865, 4.97

[[-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [0.49183608904818454, -2.3115073976360088], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [1.1473551215894577, -0.7113200917642102], [1.4855951525293154, -0.7553681338709693], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-0.5219137024228613, 3.0073875817036413], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [-2.77141485327454, -1.5161733575595002], [-8.245527927262

[[2.8189720581437347, 0.6007692231345159], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.2188459252399664], [3.5328982940662907, 4.311729332661249], [5.8760239454182335, 5.08200754778629], [4.638592672780865, 4.97

[[-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [0.49183608904818454, -2.3115073976360088], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [1.1473551215894577, -0.7113200917642102], [1.4855951525293154, -0.7553681338709693], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-0.5219137024228613, 3.0073875817036413], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [-2.77141485327454, -1.5161733575595002], [-8.245527927262

In [83]:
xs=[]
ys=[]
for data in res:
    xs.append([data[j][0] for j in range(len(data))])
    ys.append([data[j][1] for j in range(len(data))])

plt.clf()
f = plt.figure(1)
plt.scatter(xs[0], ys[0], c='r')
plt.scatter(xs[1], ys[1], c='g')
plt.scatter(xs[2], ys[2], c='b')
f.show()

In [84]:
res = kmeans(X,heuristique_centres(X,k))


[[2.8189720581437347, 0.6007692231345159], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [6.535185543874066, 0.5405683584274343], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.2188459252399664], [3.5328982940662907, 4.311729332661249], [5.8760239454182335, 5.

[[-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [-2.77141485327454, -1.5161733575595002], [-8.245527927262675, 3.3324461356100743], [-1.1808567131539882, -3.335772866520311], [-1.0395503527977943, -2.357289680608789], [-1.7624787683020522, -4.41152489902222], [-1.381728758686162, -2.6396451522802193], [-3.961661111705337, -2.9404312203496223], [-5.221896737466

[[2.8189720581437347, 0.6007692231345159], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.2188459252399664], [3.5328982940662907, 4.311729332661249], [5.8760239454182335, 5.08200754778629], [4.638592672780865, 4.97

[[-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [0.49183608904818454, -2.3115073976360088], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [2.0484791436212646, -3.919694158890705], [-2.77141485327454, -1.5161733575595002], [-8.245527927262675, 3.3324461356100743], [-1.1808567131539882, -3.335772866520311], [-1.03955035279779

[[2.8189720581437347, 0.6007692231345159], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.2188459252399664], [3.5328982940662907, 4.311729332661249], [5.8760239454182335, 5.08200754778629], [4.638592672780865, 4.97

...

[[-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [0.49183608904818454, -2.3115073976360088], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [1.1473551215894577, -0.7113200917642102], [1.4855951525293154, -0.7553681338709693], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-0.5219137024228613, 3.0073875817036413], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [-2.77141485327454, -1.5161733575595002], [-8.245527927262

[[2.8189720581437347, 0.6007692231345159], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.2188459252399664], [3.5328982940662907, 4.311729332661249], [5.8760239454182335, 5.08200754778629], [4.638592672780865, 4.97

[[-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [0.49183608904818454, -2.3115073976360088], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [1.1473551215894577, -0.7113200917642102], [1.4855951525293154, -0.7553681338709693], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-0.5219137024228613, 3.0073875817036413], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [-2.77141485327454, -1.5161733575595002], [-8.245527927262

[[2.8189720581437347, 0.6007692231345159], [4.713393106263758, 2.4423908888578643], [3.2790532147103795, 2.141228355906609], [3.614438806678783, 1.037048902907393], [4.094392807241024, 4.303214436189647], [1.5389407024120203, 5.559680030657122], [6.01601813912851, 4.123611500093395], [6.014375140490326, 1.4711380287757878], [3.725209101970023, 0.9075559337302015], [3.400984043594352, 4.185910875865213], [2.453399541773604, 2.112832143637246], [2.463435130250498, 1.1100627306425457], [2.4990480218151547, 2.9716810959095663], [1.5016181084875293, 3.524169435312067], [7.7114304972198155, 4.659261185357692], [3.5710426644182087, 1.0152012975085867], [2.3293032652203705, 6.784443536730327], [4.11360959346334, 1.5756344328994951], [4.426900001673773, 7.69631325525979], [4.26868358585409, 4.38010810923005], [4.671201240249092, 2.2123453981766263], [1.4452516436919285, 2.2188459252399664], [3.5328982940662907, 4.311729332661249], [5.8760239454182335, 5.08200754778629], [4.638592672780865, 4.97

[[-3.257353167876004, 3.3712402634441343], [-4.978334469076698, -4.937019308257924], [-5.691124962287189, -2.9913428239306117], [-4.502418105846322, 3.3739253980346087], [-2.730075072322382, -4.002314336791137], [0.49183608904818454, -2.3115073976360088], [-4.765688228146709, 0.9584353795643539], [-0.6061197985692914, -1.9725927103557135], [1.1473551215894577, -0.7113200917642102], [1.4855951525293154, -0.7553681338709693], [-4.727431386819607, 4.476954801258111], [-3.220388139972411, -1.4446149779203559], [-3.5661386683431466, -1.4548038141308643], [-1.457374633679227, -0.548694000126785], [-0.5219137024228613, 3.0073875817036413], [-2.269366971399506, -3.7573417889227447], [-4.643743701905738, -2.488672359020868], [-5.743615633481093, 0.8448227709102898], [-3.1140393955915076, -1.6953835514664717], [-2.0042294842040507, -4.122938208160005], [-3.9259619328279634, 0.5087008366409018], [-0.6169044568182154, -1.9909625627664522], [-2.77141485327454, -1.5161733575595002], [-8.245527927262

In [85]:
xs=[]
ys=[]
for data in res:
    xs.append([data[j][0] for j in range(len(data))])
    ys.append([data[j][1] for j in range(len(data))])

g = plt.figure(2)
plt.scatter(xs[0], ys[0], c='r')
plt.scatter(xs[1], ys[1], c='g')
plt.scatter(xs[2], ys[2], c='b')
g.show()